In [1]:
import QuantLib as ql

In [2]:
refDate = ql.Date(30,4,2014)

In [3]:
ql.Settings.instance().evaluationDate=refDate

In [4]:
print("EvalDate: ",ql.Settings.instance().evaluationDate)

EvalDate:  April 30th, 2014


In [5]:
forward6mLevel = 0.025
oisLevel = 0.02

In [6]:
forward6mQuote = ql.QuoteHandle(ql.SimpleQuote(forward6mLevel))
oisQuote = ql.QuoteHandle(ql.SimpleQuote(oisLevel))

In [7]:
yts6m = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), forward6mQuote, ql.Actual365Fixed() ))
ytsOis = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), oisQuote, ql.Actual365Fixed() ))

In [8]:
euribor6m = ql.Euribor6M(yts6m)

In [9]:
volLevel = 0.20

In [10]:
volQuote = ql.QuoteHandle(ql.SimpleQuote(volLevel))

In [11]:
swaptionVol = ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote,
                                                                                ql.Actual365Fixed())

In [12]:
strike = 0.04

In [13]:
effectiveDate = ql.TARGET().advance(refDate, ql.Period(2,ql.Days))

In [14]:
effectiveDate

Date(5,5,2014)

In [15]:
maturityDate = ql.TARGET().advance(effectiveDate, ql.Period(10,ql.Years))

In [16]:
maturityDate

Date(6,5,2024)

In [17]:
fixedSchedule = ql.Schedule(effectiveDate, maturityDate, ql.Period(1,ql.Years), ql.TARGET(),
                            ql.ModifiedFollowing, ql.ModifiedFollowing,
                            ql.DateGeneration.Forward, False)

In [18]:
for i in fixedSchedule:
    print(i)

May 5th, 2014
May 5th, 2015
May 5th, 2016
May 5th, 2017
May 7th, 2018
May 6th, 2019
May 5th, 2020
May 5th, 2021
May 5th, 2022
May 5th, 2023
May 6th, 2024


In [19]:
floatingSchedule = ql.Schedule(effectiveDate, maturityDate, ql.Period(6,ql.Months), ql.TARGET(),
                            ql.ModifiedFollowing, ql.ModifiedFollowing,
                            ql.DateGeneration.Forward, False)

In [20]:
for i in floatingSchedule:
    print(i)

May 5th, 2014
November 5th, 2014
May 5th, 2015
November 5th, 2015
May 5th, 2016
November 7th, 2016
May 5th, 2017
November 6th, 2017
May 7th, 2018
November 5th, 2018
May 6th, 2019
November 5th, 2019
May 5th, 2020
November 5th, 2020
May 5th, 2021
November 5th, 2021
May 5th, 2022
November 7th, 2022
May 5th, 2023
November 6th, 2023
May 6th, 2024


In [21]:
underlying = ql.NonstandardSwap(ql.VanillaSwap.Payer, [1.0]*(len(fixedSchedule)-1), [1.0]*(len(floatingSchedule)-1), fixedSchedule,
                                              [strike]*(len(fixedSchedule)-1), ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360())

In [22]:
for cf in underlying.fixedLeg():
    print (cf.date(), cf.amount())

May 5th, 2015 0.040000000000000036
May 5th, 2016 0.040000000000000036
May 5th, 2017 0.040000000000000036
May 7th, 2018 0.04022222222222216
May 6th, 2019 0.03988888888888886
May 5th, 2020 0.03988888888888886
May 5th, 2021 0.040000000000000036
May 5th, 2022 0.040000000000000036
May 5th, 2023 0.040000000000000036
May 6th, 2024 0.04011111111111121


In [23]:
for cf in underlying.floatingLeg():
    print (cf.date(), cf.amount())

November 5th, 2014 0.012682488917620917
May 5th, 2015 0.012474424851870225
November 5th, 2015 0.012682488917621138
May 5th, 2016 0.012543774790186868
November 7th, 2016 0.012821222048292169
May 5th, 2017 0.012335739224410558
November 6th, 2017 0.012751853107389575
May 7th, 2018 0.012543774790186868
November 5th, 2018 0.012543774790186868
May 6th, 2019 0.012543774790186868
November 5th, 2019 0.012613129478662223
May 5th, 2020 0.012543774790186868
November 5th, 2020 0.012682488917621138
May 5th, 2021 0.012474424851870227
November 5th, 2021 0.01268248891762114
May 5th, 2022 0.012474424851870003
November 7th, 2022 0.012821222048292391
May 5th, 2023 0.012335739224410556
November 6th, 2023 0.012751853107389573
May 6th, 2024 0.01254377479018687


In [24]:
exerciseDates = [ql.TARGET().advance(fixedSchedule[i], ql.Period(-2, ql.Days)) for i in range(1,10)]

In [25]:
exerciseDates

[Date(30,4,2015),
 Date(3,5,2016),
 Date(3,5,2017),
 Date(3,5,2018),
 Date(2,5,2019),
 Date(30,4,2020),
 Date(3,5,2021),
 Date(3,5,2022),
 Date(3,5,2023)]

In [26]:
exercise = ql.BermudanExercise(exerciseDates, False)

In [27]:
swaption = ql.NonstandardSwaption(underlying, exercise)

In [28]:
stepDates = [exerciseDates[i] for i in range(0, len(exerciseDates)-1)]

In [29]:
sigmas = [ql.QuoteHandle(ql.SimpleQuote(0.01)) for i in range(0, len(stepDates)+1)]

In [30]:
reversion = [ql.QuoteHandle(ql.SimpleQuote(0.01))]

In [31]:
gsr = ql.Gsr(yts6m, stepDates, sigmas, reversion)

In [32]:
swaptionEngine = ql.Gaussian1dNonstandardSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(),ytsOis)

In [33]:
swaption.setPricingEngine(swaptionEngine)

In [34]:
swapBase = ql.EuriborSwapIsdaFixA(ql.Period(10,ql.Years), yts6m, ytsOis)

In [35]:
basket = swaption.calibrationBasket(swapBase, swaptionVol, "Naive")

In [36]:
type(swaptionVol)

QuantLib.QuantLib.ConstantSwaptionVolatility

In [37]:
for helper in basket:
    print(helper.swaptionExpiryDate(), helper.swaptionMaturityDate(), helper.swaptionNominal(), 
          helper.swaptionStrike(),  helper.volatility().value())

April 30th, 2015 May 6th, 2024 1.0 0.025306668190455645 0.2
May 3rd, 2016 May 6th, 2024 1.0 0.025300288641156826 0.2
May 3rd, 2017 May 6th, 2024 1.0 0.025302909623471738 0.2
May 3rd, 2018 May 6th, 2024 1.0 0.02530648522594177 0.2
May 2nd, 2019 May 6th, 2024 1.0 0.025311355766312146 0.2
April 30th, 2020 May 6th, 2024 1.0 0.025300261433388174 0.2
May 3rd, 2021 May 6th, 2024 1.0 0.02530613242255889 0.2
May 3rd, 2022 May 6th, 2024 1.0 0.025317868849915624 0.2
May 3rd, 2023 May 6th, 2024 1.0 0.02535300825681316 0.2


In [38]:
for i in range(0, len(basket)):
    basket[i].setPricingEngine(swaptionEngine)

In [39]:
method = ql.LevenbergMarquardt()

In [40]:
ec = ql.EndCriteria(1000, 10, 1e-8, 1e-8, 1e-8)

In [65]:
gsr.calibrateVolatilitiesIterative(basket, method, ec, ql.NoConstraint(), [])

RuntimeError: wrong argument type

In [55]:
list(basket)

[<QuantLib.QuantLib.CalibrationHelper; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< CalibrationHelper > >::value_type *' at 0x000001A814E32690> >,
 <QuantLib.QuantLib.CalibrationHelper; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< CalibrationHelper > >::value_type *' at 0x000001A814EFBD80> >,
 <QuantLib.QuantLib.CalibrationHelper; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< CalibrationHelper > >::value_type *' at 0x000001A814EFBC90> >,
 <QuantLib.QuantLib.CalibrationHelper; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< CalibrationHelper > >::value_type *' at 0x000001A814EFBCC0> >,
 <QuantLib.QuantLib.CalibrationHelper; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< CalibrationHelper > >::value_type *' at 0x000001A814EFBCF0> >,
 <QuantLib.QuantLib.CalibrationHelper; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< CalibrationHelper > >::value_type *' at 0x000001A814E51DE0> >,
 <QuantLib

In [51]:
helper=basket[0]

In [53]:
helper

<QuantLib.QuantLib.CalibrationHelper; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< CalibrationHelper > >::value_type *' at 0x000001A814E32690> >

In [56]:
method

<QuantLib.QuantLib.LevenbergMarquardt; proxy of <Swig Object of type 'LevenbergMarquardt *' at 0x000001A814FC9660> >

In [57]:
ec

<QuantLib.QuantLib.EndCriteria; proxy of <Swig Object of type 'EndCriteria *' at 0x000001A814E51540> >

In [63]:
ql.NoConstraint()

<QuantLib.QuantLib.NoConstraint; proxy of <Swig Object of type 'NoConstraint *' at 0x000001A814A3BF00> >